# Building the Question Answering System

In [1]:
import re
import numpy as np
import pandas as pd
import sklearn
from nltk.tokenize import word_tokenize
import keras

Using TensorFlow backend.


# Preprocessing

#### Hyperparameter Notes
-Word vector size (50,100,200,300)
-how to deal with tokens with no word vectorization (REMOVE, generate random array, spell checker?)

In [2]:
train_df=pd.read_json("C:/Users/Lukas Buteliauskas/Desktop/training_data.json").reset_index(drop=True)
dev_df=pd.read_json("C:/Users/Lukas Buteliauskas/Desktop/validation_data.json").reset_index(drop=True)

## Word Vectorization
To be able to use words, phrases, questions or other natural language constructs in our model we require a to provide our neural network a numerical representation of our words (as these are the elemental NLP 'particles'). The simplest implementation would be to use 'one hot encoding' and define each word as a vector the size of our dictionary (the number of unique words found in our collection of documents, our corpus). However, this approach will most likely be insufficient for the purposes of a question answering system. word2vec and GloVe are 2 popular choices sophisticated options for word embeddings that also capture word similarities. I will not go into the details of either architecture other than to say that we will not be re-training the word vectors due to the insufficient size of the dataset, and we will begin with the GloVe word embeddings due to it's superior performance in most 'downstream' modelling tasks. Having said that, given the simplicity of swapping word vector representations we will also test out performance with word2vec (providing we can do so in a time-efficient manner).

Info and download links for GloVe can be found at: https://nlp.stanford.edu/projects/glove/

### 'Word Vector' Custom Functions

In [3]:
def get_word_vector_dict(url_or_path):
    """Takes a URL or a local path and returns a dictionary of GloVe word vectors where the key is the word and the value is the 
    word vector with the dimension specified in the input file."""
    
    with open(url_or_path, encoding="utf8") as glove_text:
        word_lines=glove_text.readlines()
    word_embeddings=[line.split(" ") for line in word_lines]
    word_vector_dict={element[0]:list(map(float, element[1:])) for element in word_embeddings}
    
    return word_vector_dict


def get_word_vector_df(url_path_or_dict):
    """Takes a URL or path like the previous function, or can take a word vector dictionary and returns a word vector dataframe.
    Rows of the dataframe are the word vectors, columns are the dimensions of the word vector, indices are the words."""
    
    if type(url_path_or_dict) is str:
        with open(url_path_or_dict, encoding="utf8") as glove_text:
            word_lines=glove_text.readlines()
        word_embeddings=[line.split(" ") for line in word_lines]
        word_vector_dict={element[0]:list(map(float, element[1:])) for element in word_embeddings}
        word_vector_df=pd.DataFrame(word_vector_dict).transpose()
    
    else:
        word_vector_df=pd.DataFrame(url_path_or_dict).transpose()
    
    return word_vector_df

### Setting up the Word Vectors
As mentioned above with regards to what model we use for the word vectors, it's important to note that the dimention of the word vectors is a hyperparameter of the Neural Networks to come, so to keep our options open we imported a few different word vectors representations and the custom functions defined above make this a 'one line of code' affair (dictionary or dataframe).


In [4]:
word_vector_50_dict=get_word_vector_dict("C:/Users/Lukas Buteliauskas/Desktop/glove.6B.50d.txt")
word_vector_50_df=get_word_vector_df(word_vector_50_dict)
vocab=np.array(word_vector_50_dict.keys()) #400k words as per the documentation.

word_vector_100_dict=get_word_vector_dict("C:/Users/Lukas Buteliauskas/Desktop/glove.6B.100d.txt")
word_vector_100_df=get_word_vector_df(word_vector_100_dict)

In [5]:
"""word_vector_200_dict=get_word_vector_dict("C:/Users/Lukas Buteliauskas/Desktop/glove.6B.200d.txt")
word_vector_200_df=get_word_vector_df(word_vector_200_dict)

word_vector_300_dict=get_word_vector_dict("C:/Users/Lukas Buteliauskas/Desktop/glove.6B.300d.txt")
word_vector_300_df=get_word_vector_df(word_vector_300_dict)"""

'word_vector_200_dict=get_word_vector_dict("C:/Users/Lukas Buteliauskas/Desktop/glove.6B.200d.txt")\nword_vector_200_df=get_word_vector_df(word_vector_200_dict)\n\nword_vector_300_dict=get_word_vector_dict("C:/Users/Lukas Buteliauskas/Desktop/glove.6B.300d.txt")\nword_vector_300_df=get_word_vector_df(word_vector_300_dict)'

## Final Pre-Processing

### Tokenization, Embedding Custom Functions

In [6]:
def get_unique_tokens(df):
    """Given a dataframe containing contexts, questions and answers, the function returns a list of unique tokens."""
    pieces_of_text=list(df["context"].unique()) + list(df["title"].unique()) + list(df["question"].unique()) 
    pieces_of_text+=list(df["answer_text"].unique())

    non_unique_tokens=[]
    for text in pieces_of_text:
        temp_tokens=word_tokenize(text)
        non_unique_tokens.append(temp_tokens)

    unique_tokens=set()
    for token in non_unique_tokens:
        unique_tokens.update(set(token))
    
    return [token.replace("``", '"').replace("''", '"').lower() for token in list(unique_tokens)]


def get_triplet(df, row_idx, cols=["context", "question", "answer_text"]):
    """Takes a dataframe and row index, and returns a (context, question, answer) triplet as a tuple."""
    triplet_list=train_df.loc[row_idx, cols].values
    return (triplet_list[0], triplet_list[1], triplet_list[2])


def tokenize(string):
    """Takes a string (either context, question or answer) and returns a list of tokens."""
    tokens=[token.replace("``", '"').replace("''", '"').lower() for token in word_tokenize(string)]
    
    split_tokens=[]
    for token in tokens:
        split_tokens.extend(re.split('(\W+)', token))
    return [token for token in split_tokens if token!=" " and token!=""]


def string_to_embedding(string, word_vector_dict=word_vector_50_dict, context=False):
    """Takes a context, question or answer and returns a list of word tokens in their vectorized form.
    If context, return a list of sequences. Updated to accept tokenizations for faster training."""
    tokens=[]
    embedding=[]
    if isinstance(string, str) is True:
        tokens=np.array(tokenize(string))
    else:
        tokens=np.array(string)
    
    if context is False:
        for token in tokens:
            if token in word_vector_dict.keys():
                embedding.extend(word_vector_dict[token])
            else:
                # We are vectorizing words with no embedding with the 'unk' token vectorization (already in GloVe)
                embedding.extend(word_vector_dict["unk"])
    else:
        """Seperate out the sentences (on the '.')"""
        dot_indeces=np.nonzero(tokens==".")[0]
        prev_dot=0
        sentences=[]
        for dot in dot_indeces:
            sentences.append(tokens[prev_dot:dot+1])
            prev_dot=dot+1
    
        """ Embed the sentences"""     
        for sentence in sentences:
            sentence_embedding=[]
            
            for token in sentence:
                if token in word_vector_dict.keys():
                    sentence_embedding.extend(word_vector_dict[token])
                else:
                    sentence_embedding.extend(word_vector_dict["unk"])
        
            embedding.append(sentence_embedding)
            sentence_embedding=[]
    
    return np.array(embedding)

The nltk tokenizer generated around 110,00 unique tokens from our contexts, questions and answers in our dataset. ~31,000 of those tokens did not have pre-trained word vectorizations in the GloVe model. Some of these tokens were numbers, expressed as strings in an unfamiliar format, some of these tokens were misspelled words, some of these tokens were works in other languages, or symbols from other alphibets and so on. 
The ideal case would be to assign to each token in our corpus a 'meaningful' word vectorization, however given the time constraint on this project, we will try to provide meaningful word vectorizations to as many of these tokens as possible through pre-processing steps following the initial word tokenization (using the nltk tokenizer).

### Tokenizing, Embedding and Padding Contexts, Questions and Answers

In [7]:
contexts=train_df.context.values
contexts_tokenized=np.array([tokenize(context) for context in train_df.context.values])
max_context_len=np.max([len(context) for context in contexts_tokenized])
print("Max number of tokens in a context:", max_context_len, "\t Timesteps=", max_context_len, " Outputs of interest=# of sent")
print("Done contexts")

questions=train_df.question.values
questions_tokenized=np.array([tokenize(question) for question in questions])
embedded_questions=np.array([string_to_embedding(tokenized_question) for tokenized_question in questions_tokenized])
max_question_len=np.max([len(question) for question in questions_tokenized])
print("Done questions")

answers=train_df.answer_text.values
answers_tokenized=np.array([tokenize(answer) for answer in answers])
embedded_answers=np.array([string_to_embedding(tokenized_answer) for tokenized_answer in answers_tokenized])
max_answer_len=np.max([len(answer) for answer in answers_tokenized])
print("Done answers")

"""question_sample=embedded_questions[0:3]
padded_questions=np.array(pad_sequences(question_sample, max_question_len*50, padding="post", dtype="float64")).reshape(3,-1,50)"""

Max number of tokens in a context: 854 	 Timesteps= 854  Outputs of interest=# of sent
Done contexts
Done questions
Done answers


'question_sample=embedded_questions[0:3]\npadded_questions=np.array(pad_sequences(question_sample, max_question_len*50, padding="post", dtype="float64")).reshape(3,-1,50)'

## Building the Dynamic Memory Network

In [8]:
from keras.models import Model
from keras.layers import Input, GRU, Masking
from keras.preprocessing.sequence import pad_sequences

In [9]:
"""Question Module"""
# question module input shape= [rows, timesteps, columns] or [observations (question-answer-context), # of words in q, 50]

question_sample=embedded_questions[0:3]
padded_questions=np.array(pad_sequences(question_sample, max_question_len*50, padding="post", dtype="float64")).reshape(3,-1,50)

question_input=Input(shape=(max_question_len,50))
question_mask=Masking(mask_value=0.0)(question_input)
question_output=GRU(units=10)(question_mask)
question_model=Model(inputs=question_input, outputs=question_output)

outputs=question_model.predict(padded_questions)
"""DMN_model = Model(inputs=[input_module, question_module], outputs=[answer_module])"""

'DMN_model = Model(inputs=[input_module, question_module], outputs=[answer_module])'

In [10]:
###### """We need to take a context and return facts, for each sentence"""
context=contexts[0]
sentence_embedding=string_to_embedding(context, context=True)
print("Sentence shape:", sentence_embedding.shape)
print("Lengths of embedded sentences:", [len(embedding) for embedding in sentence_embedding])


context_input=Input(shape=(7,50))
facts_output=GRU(units=10)(context_input)
input_model=Model(inputs=context_input, outputs=facts_output)

Sentence shape: (4,)
Lengths of embedded sentences: [1700, 2150, 1300, 2650]


In [11]:
"""def split_keep_sep(tokens, sep, only_unique=True):
    Takes a string or a list of tokens, and splits on 'sep' while keeping sep. Returns a set of unique tokens, or a list
    of tokens after splitting.
    DO NOT PASS IN A STRING OR IT WILL RETURN A BUNCH OF CHARACTERS
    split_tokens=[]
    for token in tokens:
        if only_unique==True:
            for sub_token in re.split("("+ sep + ")", token):
                if sub_token not in split_tokens:
                    split_tokens.append(sub_token)
        else: # if we want just a list of all the tokens after seperation
            split_tokens.extend(re.split("("+ sep + ")", token))
                
    return [token for token in split_tokens if token!=""]"""

"""def tokenize_further(tokens, seperator_list=["-","–","—","'","£","/",":"]):
    Further splits the tokens on the seperators, while keeping the seperators as tokens.
    Returns a further tokenized list of tokens.
    split_tokens=split_keep_sep(tokens, seperator_list[0], only_unique=False)
    for sep in seperator_list[1:]:
        split_tokens=split_keep_sep(split_tokens, sep, only_unique=False)
    
    return [token for token in split_tokens if token!=""] # filters out "" (empty space tokens)"""

"""
unique_tokens=get_unique_tokens(train_df)
no_embeddings=[token for token in unique_tokens if token not in word_vector_50_dict.keys()]
split_unique_tokens=split_keep_sep(unique_tokens, "-")
split_unique_tokens2=split_keep_sep(split_unique_tokens, "–")
split_unique_tokens3=split_keep_sep(split_unique_tokens2, "—")
split_unique_tokens4=split_keep_sep(split_unique_tokens3, "'")
split_unique_tokens5=split_keep_sep(split_unique_tokens4, "£")
split_unique_tokens6=split_keep_sep(split_unique_tokens5, "/")
split_unique_tokens7=split_keep_sep(split_unique_tokens6, ":")

print("Num of initial tokens:", len(unique_tokens), "\nNum of tokens after '-' split:", len(split_unique_tokens))
print("Num of tokens after 2nd split", len(split_unique_tokens2),"\nNum of tokens after 3rd split", len(split_unique_tokens3))
print("Num of tokens after 4th split", len(split_unique_tokens4), "\nNum of tokens after 5th split", len(split_unique_tokens5))
print("Num of tokens after 6th split", len(split_unique_tokens6), "\nNum of tokens after 7th split", len(split_unique_tokens7),"\n")

no_embeddings_2=[token for token in split_unique_tokens if token not in word_vector_50_dict.keys()]
no_embeddings_3=[token for token in split_unique_tokens2 if token not in word_vector_50_dict.keys()]
no_embeddings_4=[token for token in split_unique_tokens3 if token not in word_vector_50_dict.keys()]
no_embeddings_5=[token for token in split_unique_tokens4 if token not in word_vector_50_dict.keys()]
no_embeddings_6=[token for token in split_unique_tokens5 if token not in word_vector_50_dict.keys()]
no_embeddings_7=[token for token in split_unique_tokens6 if token not in word_vector_50_dict.keys()]
no_embeddings_8=[token for token in split_unique_tokens7 if token not in word_vector_50_dict.keys()]

print("Number of tokens with no embedding in GloVe (with the current tokenization):")
print("with '-' with words:",len(no_embeddings),"\nwith '-' seperate:", len(no_embeddings_2),)
print("with 2 - with words:", len(no_embeddings_3),"\nwith all - with words:", len(no_embeddings_4))
print("with all - and ' with words:", len(no_embeddings_5) ,"\nwith all -, ', £ with words:", len(no_embeddings_6))
print("with all -, ', £, / with words:", len(no_embeddings_7),"\nwith all -, ', £, /, : with words:", len(no_embeddings_8),"\n")

print(no_embeddings_8[0:100])"""


'\nunique_tokens=get_unique_tokens(train_df)\nno_embeddings=[token for token in unique_tokens if token not in word_vector_50_dict.keys()]\nsplit_unique_tokens=split_keep_sep(unique_tokens, "-")\nsplit_unique_tokens2=split_keep_sep(split_unique_tokens, "–")\nsplit_unique_tokens3=split_keep_sep(split_unique_tokens2, "—")\nsplit_unique_tokens4=split_keep_sep(split_unique_tokens3, "\'")\nsplit_unique_tokens5=split_keep_sep(split_unique_tokens4, "£")\nsplit_unique_tokens6=split_keep_sep(split_unique_tokens5, "/")\nsplit_unique_tokens7=split_keep_sep(split_unique_tokens6, ":")\n\nprint("Num of initial tokens:", len(unique_tokens), "\nNum of tokens after \'-\' split:", len(split_unique_tokens))\nprint("Num of tokens after 2nd split", len(split_unique_tokens2),"\nNum of tokens after 3rd split", len(split_unique_tokens3))\nprint("Num of tokens after 4th split", len(split_unique_tokens4), "\nNum of tokens after 5th split", len(split_unique_tokens5))\nprint("Num of tokens after 6th split", len(sp